In [18]:
import pandas as pd
import numpy as np

### Getting Coordinate Data

In [23]:
coordinates = pd.read_csv('Geospatial_Coordinates.csv')
coordinates.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


### Fetching and Transforming Data

In [24]:
import wikipedia as wd
html = wd.page("List of postal codes of Canada: M").html().encode("UTF-8")
df = pd.read_html(html)[0]
df_drop = df[df.Borough != "Not assigned"].reset_index(drop=True)
df_grouped = df_drop.groupby(["Postal Code", "Borough"], as_index=False).agg(lambda x : ",".join(x))
for index, row in df_grouped.iterrows():
    if row["Neighborhood"] == "Not assigned":
        row["Neighborhood"] = row["Borough"]
print(df_grouped.shape)
df_grouped.head()

(103, 3)


,Postal Code,Borough,Neighborhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


### Merging Both DataFrames to get Latitudes and Longitudes for neighborhoods

In [26]:
df_latlon = df_grouped.merge(coordinates, on="Postal Code", how="left")
df_latlon.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


### Check to make sure the coordinates are added as required by the 2nd question

In [27]:
column_names = ["PostalCode", "Borough", "Neighborhood", "Latitude", "Longitude"]
test_df = pd.DataFrame(columns=column_names)

test_list = ["M5G", "M2H", "M4B", "M1J", "M4G", "M4M", "M1R", "M9V", "M9L", "M5V", "M1B", "M5A"]

for postcode in test_list:
    test_df = test_df.append(df_latlon[df_latlon["Postal Code"]==postcode], ignore_index=True)
    
test_df

C:\Users\Anmol\Anaconda3\envs\gammavishwanathan\lib\site-packages\pandas\core\frame.py:7123: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort,


,Borough,Latitude,Longitude,Neighborhood,Postal Code,PostalCode
0,Downtown Toronto,43.657952,-79.387383,Central Bay Street,M5G,NaN
1,North York,43.803762,-79.363452,Hillcrest Village,M2H,NaN
2,East York,43.706397,-79.309937,"Parkview Hill, Woodbine Gardens",M4B,NaN
3,Scarborough,43.744734,-79.239476,Scarborough Village,M1J,NaN
4,East York,43.709060,-79.363452,Leaside,M4G,NaN
5,East Toronto,43.659526,-79.340923,Studio District,M4M,NaN
6,Scarborough,43.750072,-79.295849,"Wexford, Maryvale",M1R,NaN
7,Etobicoke,43.739416,-79.588437,"South Steeles, Silverstone, Humbergate, Jamest...",M9V,NaN
8,North York,43.756303,-79.565963,Humber Summit,M9L,NaN
9,Downtown Toronto,43.628947,-79.394420,"CN Tower, King and Spadina, Railway Lands, Har...",M5V,NaN


In [40]:
!pip install geocoder

  Created wheel for future: filename=future-0.18.2-cp36-none-any.whl size=491062 sha256=e74a942a86623b5eb67333ce47634eee98db12fe8b6d5f8db38ff41508835841
  Stored in directory: C:\Users\Anmol\AppData\Local\pip\Cache\wheels\8b\99\a0\81daf51dcd359a9377b110a8a886b3895921802d2fc1b2397e
Successfully built future


In [41]:
from geopy.geocoders import Nominatim

In [42]:
address = 'Toronto'

geolocator = Nominatim(user_agent="my-application")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


### Create a map and markers

In [44]:
import folium
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(df_latlon['Latitude'], df_latlon['Longitude'], df_latlon['Borough'], df_latlon['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_toronto)  
    
map_toronto

### Exploring neighborhoods in Toronto

In [46]:
borough_names = list(df_latlon.Borough.unique())

borough_with_toronto = []

for x in borough_names:
    if "toronto" in x.lower():
        borough_with_toronto.append(x)
        
borough_with_toronto

['East Toronto', 'Central Toronto', 'Downtown Toronto', 'West Toronto']

In [47]:
# create a new DataFrame with only boroughs that contain the word Toronto
df_latlon = df_latlon[df_latlon['Borough'].isin(borough_with_toronto)].reset_index(drop=True)
print(df_latlon.shape)
df_latlon.head()

(39, 5)


,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M4E,East Toronto,The Beaches,43.676357,-79.293031
1,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188
2,M4L,East Toronto,"India Bazaar, The Beaches West",43.668999,-79.315572
3,M4M,East Toronto,Studio District,43.659526,-79.340923
4,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790


In [48]:
# add markers to map
for lat, lng, borough, neighborhood in zip(df_latlon['Latitude'],df_latlon['Longitude'], df_latlon['Borough'], df_latlon['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_toronto)  
    
map_toronto